In [ ]:
import ast
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
import pandas as pd
from sklearn.model_selection import KFold
import joblib

In [73]:
# abrindo os dados de treinamento
df = pd.read_csv("./../ansatz_result/data.csv")
X = df.drop(columns=["target"]).to_numpy()
y = pd.DataFrame(df['target'].apply(ast.literal_eval).tolist()).to_numpy()

# y será uma lista de probabilidades com o melhor ansatz
# essa probabilidade de cada ansatz i sera definida como prob(i) = ansatz(i)/soma de todos os ansatz
y_prob = y / y.sum(axis=1, keepdims=True)

In [74]:
# criando kfold
kf = KFold(n_splits=3, shuffle=True, random_state=42)

# Definir os parâmetros a testar
param_grid = {
    'max_depth': [3, 5, 7, None],
    'min_samples_split': [2, 5, 10, 15, 20, 25, 30],
    'min_samples_leaf': [1, 2, 4, 5,  10, 15, 20, 30, 35, 40],
    'criterion': ['squared_error', 'friedman_mse']
}

dt_regressor_classifier = DecisionTreeRegressor(random_state=45)

grid_search = GridSearchCV(dt_regressor_classifier, param_grid, cv=kf, scoring='neg_mean_squared_error', n_jobs=12)

grid_search.fit(X, y_prob)

# Melhores parâmetros
print("Melhores parâmetros:", grid_search.best_params_)

Melhores parâmetros: {'criterion': 'squared_error', 'max_depth': 5, 'min_samples_leaf': 10, 'min_samples_split': 2}


In [75]:
# Usar os melhores parâmetros encontrados para validação cruzada
best_dt_regressor_classifier = grid_search.best_estimator_

score = cross_val_score(best_dt_regressor_classifier, X, y_prob, cv=kf)
print("Scores:", score)
print("Média:", score.mean())
print("Desvio padrão:", score.std())


Scores: [0.80291593 0.80676946 0.77695103]
Média: 0.7955454737531786
Desvio padrão: 0.013242037984241688


In [76]:
# treinando o dt classifier e obtendo os top3 ansatz mais classificados
best_dt_regressor_classifier.fit(X, y_prob)


# 3. Prever e rankear
def predict_rank_ansatz(model, X_new, k=3):
    probas = model.predict(X_new.reshape(1, -1))[0]
    topk = np.argsort(probas)[-k:][::-1]
    return topk, probas[topk]

# Rankear os top-3 ansatz com maiores probabilidades
top3_idx, top3_probs = predict_rank_ansatz(best_dt_regressor_classifier, X[6])

print(f"Top-3 Ansatz: {top3_idx} com probabilidades {top3_probs}")

Top-3 Ansatz: [14 11 20] com probabilidades [0.03719537 0.03694778 0.03689234]


In [77]:
# salvar o modelo treinado
joblib.dump(best_dt_regressor_classifier, './models_salvos/dt_regressor_classifier.joblib')

['./models_salvos/dt_regressor_classifier.joblib']